# This code on-server at
## /home/robbie/uit_mnt/home/romal7177/nrcs/preprocess_ak.py

In [ ]:
import scipy
from netCDF4 import Dataset
import numpy as np
from scipy.spatial import KDTree
import h5py
from ll_xy import lonlat_to_xy
import itertools
import matplotlib.pyplot as plt
import datetime
import calendar
import pandas as pd
import os
import pickle
import tqdm

server_dir=''
h5_dir = f'{server_dir}/scratch/robbie/nrcs/h5_dir/'

cycles = list(np.arange(1,36))+list(np.arange(100,183))
cycles = list(np.arange(134,183))

for cycle_num in tqdm.tqdm(cycles):

    ak_dir = f'{server_dir}/scratch/robbie/robbie_scratch/ftp-access.aviso.altimetry.fr/geophysical-data-record/saral/gdr_f/'
    cycle_code = str(cycle_num).zfill(3)
    cycle_dir = f'{ak_dir}cycle_{cycle_code}/'

    file_list = os.listdir(cycle_dir)

    file_list = [x for x in file_list if '.nc' in x]

    for file in file_list:
        
        try:

            with Dataset(f'{cycle_dir}{file}') as d:

                surf_class = np.array(d['surf_class'])
                class_40hz = np.repeat(surf_class[:,np.newaxis],40,axis=1)

                data_dict = {'lon':np.array(d['lon_40hz']).flatten(),
                             'lat':np.array(d['lat_40hz']).flatten(),
                             'pp':np.array(d['peakiness_40hz']).flatten(),
                              'sigma0':np.array(d['seaice_sig0_40hz']).flatten(),
                              'si_qual':np.array(d['seaice_qual_flag_40hz']).flatten(),
                              'lew':np.array(d['ice2_sigmal_40hz']).flatten(),
                              'class':class_40hz.flatten(),
                            }

                df = pd.DataFrame(data_dict)

            df = df[df['sigma0']<1000]

            df_nh = df[df['lat']>50]

            df_sh = df[df['lat']<-50]

            key=str(file.split('_')[4])+str(file.split('_')[5])

            df_nh.to_hdf(f'{h5_dir}ak_nh_{cycle_code}_jan25.h5',key='nh'+key,mode='a')
            df_sh.to_hdf(f'{h5_dir}ak_sh_{cycle_code}_jan25.h5',key='sh'+key,mode='a')
        
        except Exception as e:
            print(e)
            print(cycle_num, file)

# Code below does daily-gridding of the hdf files onto the relevant ice type data

In [1]:
import scipy
from netCDF4 import Dataset
import numpy as np
from scipy.spatial import KDTree
from ll_xy import lonlat_to_xy
import itertools
import matplotlib.pyplot as plt
import datetime
import calendar
import pandas as pd
import h5py
import os
import pickle
import tqdm

server_dir = '/home/robbie/uit_mnt'

satam_directory = f'{server_dir}/Data/romal7177/ResearchData/IFT/EarthObservation/SatelliteAltimetry/'
scratch=f'{server_dir}/scratch/robbie/nrcs/pickles'

nh_it_dir=f'{satam_directory}OSISAF Sea Ice Type'
f = '2019/12/ice_type_nh_polstere-100_multi_201912201200.nc'
x = Dataset(f'{nh_it_dir}/{f}')
longrid = np.array(x['lon'])
latgrid = np.array(x['lat'])
nh_xgrid,nh_ygrid=lonlat_to_xy(longrid,latgrid,hemisphere='n')
nh_tree = KDTree(list(zip(nh_xgrid.ravel(),
                       nh_ygrid.ravel())))

sh_it_dir = f'{server_dir}/scratch/robbie/melsheimer_ice_type'
latf = 'south_lat_12km.hdf'
sh_lat = Dataset(f'{sh_it_dir}/{latf}')
v = list(sh_lat.variables)[0]
lat = np.array(sh_lat[v])

lonf = 'south_lon_12km.hdf'
sh_lon = Dataset(f'{sh_it_dir}/{lonf}')
v = list(sh_lon.variables)[0]
lon = np.array(sh_lon[v])

sh_xgrid,sh_ygrid=lonlat_to_xy(lon,lat,hemisphere='s')
sh_tree = KDTree(list(zip(sh_xgrid.ravel(),
                       sh_ygrid.ravel())))

trees = {'s':sh_tree,'n':nh_tree}
grids = {'s':sh_xgrid,'n':nh_xgrid}

def get_date_from_key(key):
    datestr = key[2:10]
    dt = datetime.date(year=int(datestr[:4]),
                       month=int(datestr[4:6]),
                       day=int(datestr[6:8]))
    return dt

ak_directory=f'{server_dir}/scratch/robbie/nrcs/h5_dir'
ak_files = os.listdir(ak_directory)
ak_files = sorted([a for a in ak_files if a[:2]=='ak'])

for file in ak_files:
    
    hem = file.split('_')[1][0]
    cycle = int(file.split('_')[2])

    print(cycle,hem)

    with h5py.File(f'{ak_directory}/{file}','r') as x:
        keys = list(x.keys())
    dates = np.array([get_date_from_key(key) for key in keys])
    dates_mask = np.ones(len(dates))
    dates_set = sorted(list(set(dates)))

    mean_grids = []

    for dt in tqdm.tqdm(dates_set):

        mean_vals = np.full(grids[hem].shape,np.nan)

        r=np.argwhere(dates==dt)

        if r.shape[0]:

            keys_on_date = keys[r[0][0]:r[-1][0]]


            basic_dict = {(a,b):[] for a,b in itertools.product(np.arange(grids[hem].shape[0]),
                                                                np.arange(grids[hem].shape[1]))}

            for key in keys_on_date:

                df = pd.read_hdf(f'{ak_directory}/{file}',mode='r',key=key)

                df = df[df['pp']<5]
                df = df[df['lew']<2]

                at_x, at_y = lonlat_to_xy(np.array(df['lon']),
                                          np.array(df['lat']),
                                          hemisphere=hem)

                tree = trees[hem]

                dist,ind = tree.query(np.array([at_x,at_y]).T)

                ind2d = np.unravel_index(ind, grids[hem].shape)

                sig0_lin = 10 ** (df['sigma0']/10)

                for (a,b,s) in zip(ind2d[0],ind2d[1],sig0_lin):
                    basic_dict[(a,b)].append(s)


            vals,xscat,yscat = [],[],[]

            for key, value in basic_dict.items():


                if value:
                    mean_vals[key[0],key[1]] = np.nanmean(value)

        mean_grids.append(mean_vals)


    pickle.dump(mean_grids,open(f'{scratch}/ak_{cycle}_{hem}h.p','wb'))
    pickle.dump(dates_set,open(f'{scratch}/ak_{cycle}_{hem}h_dates.p','wb'))

In [4]:
hem='n'
pickles = os.listdir(f'{scratch}')
pickles = sorted([x for x in pickles if ((x[:2]=='ak')&(f'{hem}h_dates.p' in x))])


dates_dict = {}
for p in pickles[:]:
    
    print(p)
    
    dates_set = pickle.load(open(f'{scratch}/{p}','rb'))
    
    dates_dict[p]=dates_set
    
    

# Old code below

In [ ]:
# prep month year pairs

winter_months=[1,2,3,4,10,11,12]
top_data = {}

mon_yr_pairs = []

for year in np.arange(2019,2023):
    for month in winter_months:
        mon_yr_pairs.append((year,month))
for month in [10,11,12]:
    mon_yr_pairs.append((2018,month))
for month in [1,2,3,4]:
    mon_yr_pairs.append((2023,month))
    
# Run code

for year,month in mon_yr_pairs:
    
    mean_grids = []

    days_in_month = calendar.monthrange(year,month)[1]

    for day in np.arange(1,days_in_month+1):
        print(day)
        date = datetime.date(year,month,day)

        # Format it into a string
        date_str_ak = f'{date.year}_{str(date.month).zfill(2)}_{str(date.day).zfill(2)}'

        ak_files_on_day = [x for x in ak_files if date_str_ak in x]

        basic_dict = {(a,b):[] for a,b in itertools.product(np.arange(xgrid.shape[0]),
                                                            np.arange(xgrid.shape[1]))}

        for f in ak_files_on_day:

            ak = scipy.io.loadmat(f'{ak_directory}/{f}')
            PP = ak['parameters'][:,20]
            lew = ak['parameters'][:,16]/(2.381e-9)
            at_lons = np.array(ak['lon'][:,0])
            at_lats = np.array(ak['lat'][:,0])
            sig0db = np.array(ak['sig0'][:,0])
            sig0_lin = 10 ** (sig0db/10)

            df = pd.DataFrame({'lon':at_lons,
                               'lat':at_lats,
                               'lew':lew,
                               'PP':PP,
                               'sig0':sig0_lin})
            df.dropna(inplace=True)
        #     # Supp of https://doi.org/10.1002/2015GL064823
            df = df[df['lew']<2]
            df = df[df['PP']<5]



            at_x, at_y = lonlat_to_xy(np.array(df['lon']),
                                      np.array(df['lat']),
                                      hemisphere='n')
            dist,ind = tree.query(np.array([at_x,at_y]).T)

            ind2d = np.unravel_index(ind, xgrid.shape)

            for (a,b,s) in zip(ind2d[0],ind2d[1],df['sig0']):
                basic_dict[(a,b)].append(s)

        vals,xscat,yscat = [],[],[]

        mean_vals = np.full(xgrid.shape,np.nan)
        for key, value in basic_dict.items():


            if value:
                mean_vals[key[0],key[1]] = np.nanmean(value)

        mean_grids.append(mean_vals)

    pickle.dump(mean_grids,open(f'{scratch}ak_{month}_{year}.p','wb'))